In [356]:
import numpy as np
import os
from six.moves import urllib

# Brute data from different sources:
# https://ai-project-inflation.s3.amazonaws.com/
DOWNLOAD_ROOT = "https://ai-project-inflation.s3.amazonaws.com/"
INFLATION_PATH = os.path.join("datasets", "inflation")

def fetch_housing_data(file_name):
    inflation_url= DOWNLOAD_ROOT+file_name
    inflation_path=INFLATION_PATH
    if not os.path.isdir(inflation_path):
        os.makedirs(inflation_path)
    inflation_path = os.path.join(inflation_path, file_name)
    urllib.request.urlretrieve(inflation_url, inflation_path)

def load_inflation_data(file_name):
    fetch_housing_data(file_name)
    return os.path.join(INFLATION_PATH,file_name)

In [395]:
import pandas as pd
import datetime
def truncate_date(date):
  ts = pd.to_datetime(str(date))
  return ts.strftime('%Y-%m')

def parsing_string_to_date_withdash(date):
  format = "%B-%Y"
  return truncate_date(datetime.datetime.strptime(date, format))

def parsing_string_to_date(date):
  format = "%B %Y"
  return truncate_date(datetime.datetime.strptime(date, format))

In [370]:
brent_crude_prices = pd.read_excel(load_inflation_data("Brent_crude_prices.xls"), sheet_name=1, header=2)
brent_crude_prices["Date"] = brent_crude_prices["Date"].apply(truncate_date)
brent_crude_prices.rename(columns={'Europe Brent Spot Price FOB (Dollars per Barrel)': 'Crude prices'}, inplace=True)
brent_crude_prices.head()
# brent_crude_prices.info()

,Date,Crude prices
0,1987-05,18.58
1,1987-06,18.86
2,1987-07,19.86
3,1987-08,18.98
4,1987-09,18.31


In [359]:
!pip install openpyxl
prime_rate = pd.read_excel(load_inflation_data("Prime_Rate_History-WOWA.xlsx"))
prime_rate["Date"] = prime_rate["Date"].apply(parsing_string_to_date)
prime_rate = prime_rate[["Date","Prime Rate" ]]
prime_rate.head()

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


,Date,Prime Rate
0,1935-01,5.50
1,1936-01,5.21
2,1937-01,5.00
3,1938-01,5.00
4,1939-01,5.00


In [360]:
bond_rate = pd.read_csv(load_inflation_data("StlouisfedCAD10yearbondrate.csv"))
bond_rate.rename(columns={'DATE': 'Date'}, inplace=True)
bond_rate["Date"] = bond_rate["Date"].apply(truncate_date)
bond_rate.head()

,Date,IRLTLT01CAM156N
0,2000-01,6.491500
1,2000-02,6.338571
2,2000-03,6.028261
3,2000-04,5.924211
4,2000-05,6.215000


In [361]:
fx_rates = pd.read_csv(load_inflation_data("FX_RATES_MONTHLY_CSV.csv"))
fx_rates["Date"] = fx_rates["Date"].apply(truncate_date)
fx_rates.head()

,Date,1 USD
0,1990-01,1.172038
1,1990-02,1.196479
2,1990-03,1.179986
3,1990-04,1.164095
4,1990-05,1.174700


In [362]:
manufacturing_rate = pd.read_csv(load_inflation_data("Stlouisfeb_hourmanurate.csv"))
manufacturing_rate.rename(columns={'DATE': 'Date'}, inplace=True)
manufacturing_rate["Date"] = manufacturing_rate["Date"].apply(truncate_date)
manufacturing_rate.head()

,Date,LCEAMN01CAM189S
0,2000-01,18.25
1,2000-02,18.33
2,2000-03,18.23
3,2000-04,18.27
4,2000-05,18.42


In [363]:
money_supply = pd.read_csv(load_inflation_data("Stlousisfed_Canadain_Money_supply.csv"))
money_supply.rename(columns={'DATE': 'Date'}, inplace=True)
money_supply["Date"] = money_supply["Date"].apply(truncate_date)
money_supply.head()

,Date,MANMM101CAM189S
0,2000-01,2.166880e+11
1,2000-02,2.216730e+11
2,2000-03,2.250820e+11
3,2000-04,2.284850e+11
4,2000-05,2.281010e+11


In [364]:
fed_fx_rates = pd.read_csv(load_inflation_data("stlousisfedFXRates.csv"))
fed_fx_rates.rename(columns={'DATE': 'Date'}, inplace=True)
fed_fx_rates["Date"] = fed_fx_rates["Date"].apply(truncate_date)
fed_fx_rates.head()

,Date,CCUSMA02CAM618N
0,2000-01,1.448226
1,2000-02,1.450341
2,2000-03,1.460354
3,2000-04,1.468305
4,2000-05,1.494781


In [408]:
cpi_inflation = pd.read_csv(load_inflation_data("1810025601-CPI_and_Inflation_ydata.csv"))
cpi_inflation["Date"] = cpi_inflation["Date"].apply(truncate_date)
cpi_inflation.head()

HTTPError: HTTP Error 403: Forbidden

In [366]:
tsx_data = pd.read_csv(load_inflation_data("TSX_data_for_20_years.csv"))
tsx_data["Date"] = tsx_data["Date"].apply(truncate_date)
tsx_data = tsx_data[["Date", "Close"]]
tsx_data.head()

,Date,Close
0,2000-01,8481.099609
1,2000-02,9129.000000
2,2000-03,9462.400391
3,2000-04,9347.599609
4,2000-05,9252.000000


In [405]:
dataList = [cpi_inflation, prime_rate, brent_crude_prices,bond_rate, fx_rates, manufacturing_rate, money_supply, fed_fx_rates, tsx_data]
data_merged = pd.DataFrame()
initialIteration = True
for single_data in dataList:
  if initialIteration:
    data_merged = single_data.copy()
    initialIteration = False
  else:
    data_merged= data_merged.merge(single_data, left_index=True, right_index=True, how='outer')
    data_merged.pop('Date_y')
    data_merged.rename(columns={'Date_x': 'Date'}, inplace=True)

In [381]:
# data_merged.to_csv('final_result.csv', index = True)
data_merged.shape


In [406]:
data_merged.head()

,Date,Inflationrate,"Measure of core inflation based on a weighted median approach, CPI-median (year-over-year percent change) 2 3 6 7","Measure of core inflation based on a trimmed mean approach, CPI-trim (year-over-year percent change) 2 3 6 8",CPI,"Consumer Price Index (CPI), all-items excluding eight of the most volatile components as defined by the Bank of Canada 9","Consumer Price Index (CPI), all-items excluding the effect of indirect taxes 10","Consumer Price Index (CPI), all-items excluding eight of the most volatile components as defined by the Bank of Canada and excluding the effect of changes in indirect taxes, seasonally adjusted 9","Consumer Price Index (CPI), all-items excluding eight of the most volatile components as defined by the Bank of Canada, seasonally adjusted 9","Consumer Price Index (CPI), all-items excluding the effect of indirect taxes, seasonally adjusted 10",Prime Rate,Crude prices,IRLTLT01CAM156N,1 USD,LCEAMN01CAM189S,MANMM101CAM189S,CCUSMA02CAM618N,Close
0,2000-05,1.6,1.4,1.6,95.5,95.5,95.4,95.4,95.4,95.3,5.50,18.58,6.491500,1.172038,18.25,2.166880e+11,1.448226,8481.099609
1,2000-06,1.7,1.6,1.8,95.8,95.8,96.0,95.6,95.7,95.8,5.21,18.86,6.338571,1.196479,18.33,2.216730e+11,1.450341,9129.000000
2,2000-07,1.8,1.6,1.8,95.8,95.9,96.3,95.7,95.7,96.1,5.00,19.86,6.028261,1.179986,18.23,2.250820e+11,1.460354,9462.400391
3,2000-08,1.9,1.7,1.9,96.0,96.0,96.2,95.9,95.9,96.1,5.00,18.98,5.924211,1.164095,18.27,2.284850e+11,1.468305,9347.599609
4,2000-09,1.8,1.6,1.8,96.2,96.2,96.6,96.0,96.0,96.5,5.00,18.31,6.215000,1.174700,18.42,2.281010e+11,1.494781,9252.000000
